In [24]:
using GaussianProcesses, Plots, SumProductNetworks, StatsFuns, Distributions
using PlotRecipes
using TestImages, Images
import SumProductNetworks.add!

#### include local code

In [41]:
include("utilFunctions.jl")
include("dataTypes.jl")
include("dataTypeUtilFunctions.jl")
include("computationFunctions.jl")
include("regionGraph.jl")
include("regionGraphUtils.jl")
include("gpUtils.jl")

mcmcKernel (generic function with 1 method)

# Load data

In [26]:
datapath = "../data/clean/motor.csv"
(data, header) = readcsv(datapath, header = true)
headerDict = Dict(col[2] => col[1] for col in enumerate(header))
X = convert(Vector, data[:,headerDict["times"]])
y = convert(Vector, data[:,headerDict["accel"]]);
y /= maximum(y);
scatter(X, y)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 50 
 
 
 -1.5 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

# Construct a region graph

Set some parameters to split up the data using Poon & Domingos Method

In [4]:
# split size
δ = 10

# data range
minX = 0
maxX = 60

60

In [5]:
(rootRegion, sumRegions, gpRegions, allPartitions) = poonDomingos(δ, minX, maxX);

In [6]:
# set IDs for convenients
RegionIDs = Dict(r[2] => r[1] for r in enumerate(union(sumRegions, gpRegions)));
PartitionIDS = Dict(p[2] => p[1] + maximum(values(RegionIDs)) for p in enumerate(allPartitions));

In [7]:
println("Found $(length(sumRegions)) regions with sum nodes")
println("Found $(length(gpRegions)) regions with gp nodes")
println("Found $(length(allPartitions)) partitions with splitting nodes")

Found 15 regions with sum nodes
Found 6 regions with gp nodes
Found 35 partitions with splitting nodes


## Construct SPN from Region Graph

Set some parameters

In [8]:
global gID = 1

numGPs = 100
numSums = 1
meanFunction = MeanZero();
kernelFunction = SE(log(5.0),log(1.0));
noise = -1.;

# put some priors on the parameters
set_priors!(kernelFunction, [Normal(log(5.0),1.0), Normal(log(1.0),1.0)])

2-element Array{Distributions.Normal{Float64},1}:
 Distributions.Normal{Float64}(μ=1.60944, σ=1.0)
 Distributions.Normal{Float64}(μ=0.0, σ=1.0)    

In [9]:
root = convertToSPN(rootRegion, gpRegions, X, y, meanFunction, kernelFunction, noise; sampleNoise = false)

BasicMCJob:
  Variable [1]: p (BasicContMuvParameter)
  GenericModel: 1 variables, 0 dependencies (directed graph)
  HMC sampler: number of leaps = 10, leap step = 0.1
  VanillaMCTuner: period = 100, verbose = false
  BasicMCRange: number of steps = 1100, burnin = 100, thinning = 1BasicMCJob:
  Variable [1]: p (BasicContMuvParameter)
  GenericModel: 1 variables, 0 dependencies (directed graph)
  HMC sampler: number of leaps = 10, leap step = 0.1
  VanillaMCTuner: period = 100, verbose = false
  BasicMCRange: number of steps = 1100, burnin = 100, thinning = 1BasicMCJob:
  Variable [1]: p (BasicContMuvParameter)
  GenericModel: 1 variables, 0 dependencies (directed graph)
  HMC sampler: number of leaps = 10, leap step = 0.1
  VanillaMCTuner: period = 100, verbose = false
  BasicMCRange: number of steps = 1100, burnin = 100, thinning = 1BasicMCJob:
  Variable [1]: p (BasicContMuvParameter)
  GenericModel: 1 variables, 0 dependencies (directed graph)
  HMC sampler: number of leaps = 10, le

Gaussian Process Sum Node [ID: 52661, 
	 w_prior: [0.2, 0.2, 0.2, 0.2, 0.2], 
	 w_posterior: [0.2, 0.2, 0.2, 0.2, 0.2]]

# Posterior Inference over SPN-GP

In [21]:
function spn_mcmc(spn::GPSumNode, gpNodes::Vector{GPLeaf};
              sampler::Klara.MCSampler=Klara.HMC(),
              nIter::Int = 1000,
              burnin::Int = 0,
              thin::Int = 1)
    
    gps = map(n -> n.gp, gpNodes)
    
    function logpost(hyp::Vector{Float64})  #log-target
        
        for (i, gp) in enumerate(gps)
            p = get_params(gp)
            e = (i*2)
            s = s-1
            p[2:3] = hyp[s:e]
            set_params!(gp, p)
            update_target!(gp)
        end
        
        dirty!(spn)
        spn_update!(spn)
        return spn_posterior(spn)
    end

    function dlogpost(hyp::Vector{Float64}) #gradient of the log-target
        Kgrad_buffer = Array{Float64}(gp.nobsv, gp.nobsv)
        for (i, gp) in enumerate(gps)
            p = get_params(gp)
            e = (i*2)
            s = s-1
            p[2:3] = hyp[s:e]
            set_params!(gp, p)
            update_target_and_dtarget!(gp, noise=false, mean=false, kern=true)
            println(gp.dtarget)
        end
        
        dirty!(spn)
        spn_update!(spn)
        
        return gp.dtarget
    end
    
    start = reduce(vcat, map(gp -> get_params(gp)[2:3], gps))
    starting = Dict(:p=>start)
    q = BasicContMuvParameter(:p, logtarget=logpost, gradlogtarget=dlogpost) 
    model = likelihood_model(q, false)                               #set-up the model
    job = BasicMCJob(model, sampler, BasicMCRange(nsteps=nIter, thinning=thin, burnin=burnin), starting)   #set-up MCMC job
    print(job)                                             #display MCMC set-up for the user
    
    run(job)                          #Run MCMC
    chain = Klara.output(job)         # Extract chain
    #set_params!(gp,start)      #reset the parameters stored in the GP to original values
    return chain
end 

spn_mcmc (generic function with 1 method)

In [49]:
nodes = SumProductNetworks.order(root)

LoadError: [91mInterruptException:[39m

In [23]:
leafs = filter(n -> isa(n, GPLeaf), SumProductNetworks.order(root))

LoadError: [91mInterruptException:[39m

In [22]:
spn_mcmc(root)

LoadError: [91mMethodError: no method matching spn_mcmc(::GPSumNode)[0m
Closest candidates are:
  spn_mcmc(::GPSumNode, [91m::Array{GPLeaf,1}[39m; sampler, nIter, burnin, thin) at In[21]:7[39m

## posterior inference of weights

In [28]:
spn_update!(root)

-40.15978046284878

In [29]:
spn_posterior(root)

-38.44777593617354

In [27]:
dirty!(root)

In [20]:
root

Gaussian Process Sum Node [ID: 52661, 
	 w_prior: [0.2, 0.2, 0.2, 0.2, 0.2], 
	 w_posterior: [0.2, 0.2, 0.2, 0.2, 0.2]]

## Plotting

In [48]:
S = 100

x = linspace(minimum(X),maximum(X),S)
xx = reduce(hcat, x for k in 1:S)
yy = reduce(hcat, spn_randIndep(root, x) for k in 1:S)

gp = GP(reshape(X, 1, length(X)), y, MeanZero(), SE(log(5.0),log(1.0)), -1.)
μ, σ = predict_y(gp,linspace(minimum(X),maximum(X),100));

#plt = scatter(X, y, label = "observations", legend = false, size = (1024, 768))
#scatter!(xx+(rand(size(xx)) * 0.3), yy, markersize = 2, markerstrokewidth = 0, label = "Posterior samples from SPN-GP");
#plot!(linspace(minimum(X),maximum(X),100), w =4, μ + 2*σ, color = :green, label = "Full GP mean + 2*std")
#plot!(linspace(minimum(X),maximum(X),100), w =4, μ - 2*σ, color = :green, label = "Full GP mean - 2*std")
#vline!([minimum(map(r -> r.min, gpRegions))], c = :orange)
#vline!(map(r -> r.max, gpRegions), c = :orange)

In [43]:
savefig("../plots/poonDomingos_sharedNoiseModel_scatter.png")

## Histogram over parameters

## Graph Plotting

In [44]:
sources = Int[]
destinations = Int[]

# add edges from regions to partitions
for r in filter(r -> isa(r, SumRegion), union(sumRegions, gpRegions))
    for p in r.partitions
        push!(sources, RegionIDs[r])
        push!(destinations, PartitionIDS[p])
    end
end

# add edges from partitions to regions
for p in allPartitions
    for r in p.regions
        push!(sources, PartitionIDS[p])
        push!(destinations, RegionIDs[r])
    end
end

In [45]:
nodenames = vcat(map(r -> string("+"),  union(sumRegions, gpRegions)), map(r -> string("*"), allPartitions));
colorings = vcat(map(r -> isa(r, SumRegion) ? :lightblue : :lightgreen,  union(sumRegions, gpRegions)), map(r -> :black, allPartitions));

In [47]:
#gr(size = (400, 400))
#graphplot(sources, destinations, names = nodenames, method = :tree, fontsize = 7, nodesize = 3, m = colorings, nodeshape = :rect)